In [1]:
import sys
!{sys.executable} -m pip install xarray zarr 'itk>=5.1rc2' dask[array] toolz itkwidgets

In [1]:
import itk
import xarray as xr
import numpy as np
from numcodecs import Blosc, blosc
import zarr
from urllib.request import urlretrieve
import os

from itkwidgets import view, compare

In [3]:
ccf_filename = 'average_template_10.nrrd'
if not os.path.exists(ccf_filename):
    url = 'http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_10.nrrd'
    urlretrieve(url, ccf_filename)

In [4]:
image = itk.imread(ccf_filename)

In [5]:
view(image, vmax=300, gradient_opacity=0.1, units='μm')

Viewer(geometries=[], gradient_opacity=0.1, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy o…

In [6]:
print(itk.size(image))
print(itk.spacing(image))

itkSize3 ([1320, 800, 1140])
itkVectorD3 ([10, 10, 10])


In [7]:
# Available in ITK 5.1 RC 2 and later
image_da = itk.xarray_from_image(image)
image_da

<xarray.DataArray (z: 1140, y: 800, x: 1320)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint16)
Coordinates:
  * x        (x) float64 0.0 10.0 20.0 30.0 ... 1.317e+04 1.318e+04 1.319e+04
  * y        (y) float64 0.0 10.0 20.0 30.0 ... 7.97e+03 7.98e+03 7.99e+03
  * z        (z) float64 0.0 10.0 20.0 30.0 ... 1.137e+04 1.138e+04 1.139e+04
Attributes:
    direction:  [[1. 0. 0.]\n [0. 1. 0.]\n [0. 0. 1.]]

In [8]:
units = 'μm'
image_da.attrs['units'] = units
image_da.attrs

{'direction': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]), 'units': 'μm'}

In [9]:
# multi-resolution pyramid
pyramid = [image_da]
reduced = image
while not np.all(np.array(itk.size(reduced)) < 64):
    level = len(pyramid)
    shrink_factors = [2**level]*3
    reduced = itk.bin_shrink_image_filter(image, shrink_factors=shrink_factors)
    reduced_da = itk.xarray_from_image(reduced)
    reduced_da.attrs['units'] = units
    print('level', level)
    print('origin', itk.origin(reduced))
    print('spacing', itk.spacing(reduced))
    print('size', itk.size(reduced))
    pyramid.append(reduced_da)

level 1
origin itkPointD3 ([5, 5, 5])
spacing itkVectorD3 ([20, 20, 20])
size itkSize3 ([660, 400, 570])
level 2
origin itkPointD3 ([15, 15, 15])
spacing itkVectorD3 ([40, 40, 40])
size itkSize3 ([330, 200, 285])
level 3
origin itkPointD3 ([35, 35, 35])
spacing itkVectorD3 ([80, 80, 80])
size itkSize3 ([165, 100, 142])
level 4
origin itkPointD3 ([75, 75, 75])
spacing itkVectorD3 ([160, 160, 160])
size itkSize3 ([82, 50, 71])
level 5
origin itkPointD3 ([155, 155, 155])
spacing itkVectorD3 ([320, 320, 320])
size itkSize3 ([41, 25, 35])


In [10]:
compare(image, itk.image_from_xarray(pyramid[-1]), mode='y', vmax=300)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [2]:
dataset_name = 'allen_ccfv3'
store_name = 'allen_ccfv3_average_template_10.zarr'

In [12]:
image_ds = image_da.to_dataset(name=dataset_name)
image_ds

<xarray.Dataset>
Dimensions:      (x: 1320, y: 800, z: 1140)
Coordinates:
  * x            (x) float64 0.0 10.0 20.0 ... 1.317e+04 1.318e+04 1.319e+04
  * y            (y) float64 0.0 10.0 20.0 30.0 ... 7.97e+03 7.98e+03 7.99e+03
  * z            (z) float64 0.0 10.0 20.0 ... 1.137e+04 1.138e+04 1.139e+04
Data variables:
    allen_ccfv3  (z, y, x) uint16 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0

In [5]:
pyramid_group_paths = [""]
for level in range(1, len(pyramid)):
#for level in range(1, 6):
    pyramid_group_paths.append('level_{0}'.format(level))
pyramid_group_paths

['', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5']

In [14]:
store = zarr.DirectoryStore(store_name)

# NOSHUFFLE since we will be visualizing with WebAssembly, which does not currently have support for intrinsics
blosc.use_threads = False
# Crashing (?)
# compressor = Blosc(cname='zstd', clevel=5, shuffle=Blosc.NOSHUFFLE)
compressor = Blosc(clevel=5, shuffle=Blosc.NOSHUFFLE)
chunk_size = 64

In [15]:
image_ds.to_zarr(store,
                 mode='w',
                 compute=True,
                 encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

In [17]:
for level in range(1, len(pyramid)):
    print('level', level)
    ds = pyramid[level].to_dataset(name=dataset_name)
    print(ds)
    compressor = Blosc(clevel=5, shuffle=Blosc.NOSHUFFLE)
    ds.to_zarr(store,
               mode='w',
               group=pyramid_group_paths[level],
               compute=True,
               encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

level 1
<xarray.Dataset>
Dimensions:      (x: 660, y: 400, z: 570)
Coordinates:
  * x            (x) float64 5.0 25.0 45.0 ... 1.314e+04 1.316e+04 1.318e+04
  * y            (y) float64 5.0 25.0 45.0 ... 7.945e+03 7.965e+03 7.985e+03
  * z            (z) float64 5.0 25.0 45.0 ... 1.134e+04 1.136e+04 1.138e+04
Data variables:
    allen_ccfv3  (z, y, x) uint16 1 1 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0
level 2
<xarray.Dataset>
Dimensions:      (x: 330, y: 200, z: 285)
Coordinates:
  * x            (x) float64 15.0 55.0 95.0 ... 1.31e+04 1.314e+04 1.318e+04
  * y            (y) float64 15.0 55.0 95.0 ... 7.895e+03 7.935e+03 7.975e+03
  * z            (z) float64 15.0 55.0 95.0 ... 1.13e+04 1.134e+04 1.138e+04
Data variables:
    allen_ccfv3  (z, y, x) uint16 0 0 0 0 0 0 0 0 0 0 0 ... 1 2 2 2 2 2 2 2 2 2
level 3
<xarray.Dataset>
Dimensions:      (x: 165, y: 100, z: 142)
Coordinates:
  * x            (x) float64 35.0 115.0 195.0 ... 1.3e+04 1.308e+04 1.316e+04
  * y            (y) float6

In [18]:
# After all modifications to the store are complete, consolidate the metadata so it is 'cloud-ready'.
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>

In [19]:
ds = xr.open_zarr(store_name, group='level_3', consolidated=True)

In [20]:
da = ds[dataset_name]
da

<xarray.DataArray 'allen_ccfv3' (z: 142, y: 100, x: 165)>
dask.array<zarr, shape=(142, 100, 165), dtype=uint16, chunksize=(64, 64, 64), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 35.0 115.0 195.0 275.0 ... 1.3e+04 1.308e+04 1.316e+04
  * y        (y) float64 35.0 115.0 195.0 ... 7.795e+03 7.875e+03 7.955e+03
  * z        (z) float64 35.0 115.0 195.0 ... 1.116e+04 1.124e+04 1.132e+04
Attributes:
    direction:  [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
    units:      μm

In [22]:
image_level_3 = itk.image_from_xarray(da)
view(image_level_3)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

In [6]:
for level in range(1, len(pyramid)):
#for level in range(1, 6):
    print('level', level)
    store = zarr.DirectoryStore(store_name + '/' + pyramid_group_paths[level])
    # Also consolidate the metadata on the pyramid levels so they can be used independently
    zarr.consolidate_metadata(store)

level 1
level 2
level 3
level 4
level 5
